In [1]:
# Python Imports
import ipywidgets as widgets
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')

# Project Imports
from src.mesh_analysis.readers.file_reader import FileReader
from src.mesh_analysis.tracers.waku_tracer import WakuTracer
from src.mesh_analysis.visualizer.waku_visualizer import add_elapsed_time

In [2]:
tracer = WakuTracer()
reader = FileReader("../../../test/log_test/", tracer)

df = reader.read()
df = add_elapsed_time(df)

In [3]:
df

sender_peer_id receiver_peer_id  \
msg_hash timestamp                                                          
0x1      2024-04-22 14:06:58.001000+00:00              A                B   
         2024-04-22 14:06:58.002000+00:00              B                D   
         2024-04-22 14:06:58.003000+00:00              A                C   
         2024-04-22 14:06:58.003000+00:00              D                E   
         2024-04-22 14:06:58.006000+00:00              A                F   
0x2      2024-04-22 14:06:59.001000+00:00              A                B   

                                           elapsed_time  
msg_hash timestamp                                       
0x1      2024-04-22 14:06:58.001000+00:00             0  
         2024-04-22 14:06:58.002000+00:00             1  
         2024-04-22 14:06:58.003000+00:00             2  
         2024-04-22 14:06:58.003000+00:00             2  
         2024-04-22 14:06:58.006000+00:00             5  
0x2      2024-04-22 14:06:59.001000+00:00             0

In [18]:
%matplotlib
output = widgets.Output()

msg_dropdown = widgets.SelectMultiple(
                        options=df.index.get_level_values(0).drop_duplicates(),
                        description='Msg hash',
                        disabled=False,
                        layout={'height':'100px', 'width':'100%'}) 

timestamp_dropdown = widgets.SelectMultiple(
                        options=[],
                        description='Timestamp',
                        disabled=False,
                        layout={'height':'100px', 'width':'100%'})

@output.capture()
def update_timestamps(change):
    selected_hashes = change['new']
    timestamps = df.loc[selected_hashes[0]].index.get_level_values('timestamp').drop_duplicates()
    timestamp_dropdown.options = timestamps

        
msg_dropdown.observe(update_timestamps, names='value')

    
fig = plt.figure(figsize=(10,10))

ax_ = fig.add_subplot(1, 1, 1)
ax_.set_xlim(-3, 3)
ax_.set_ylim(-3, 3)

data_prepared = df.reset_index()
complete_graph = nx.from_pandas_edgelist(data_prepared, 'sender_peer_id', 'receiver_peer_id', edge_attr=['timestamp', 'msg_hash', 'elapsed_time'], create_using=nx.DiGraph)
g_pos_layout = nx.kamada_kawai_layout(complete_graph)


def test(x, y, data_):
    # todo maintain limits?
    ax_.clear()
    ax_.set_xlim(-2, 2)
    ax_.set_ylim(-2, 2)

    data_ = data_[data_.index.get_level_values(0) == x[0]]
    first_timestamp = data_.index.get_level_values(1)[0]    
    df_ = data_.loc[(x[0], first_timestamp,):(x[0], y[0])]
    df_.reset_index(inplace=True)
    
    selected_graph = nx.from_pandas_edgelist(df_, 'sender_peer_id', 'receiver_peer_id', edge_attr=['timestamp', 'msg_hash', 'elapsed_time'], create_using=nx.DiGraph)
 
    g_pos_layout_selected = {key: g_pos_layout[key] for key in list(selected_graph.nodes)}

    nx.draw(selected_graph, g_pos_layout_selected, ax=ax_, with_labels=True)
    edge_labels = {} 
    for u, v, data in selected_graph.edges(data=True):
        edge_labels[u, v] = f"{data['elapsed_time']}ms"
        
    nx.draw_networkx_edge_labels(selected_graph, g_pos_layout_selected, edge_labels=edge_labels, ax=ax_)
    plt.show()

i_plot = widgets.interactive(test, x=msg_dropdown, y=timestamp_dropdown, data_=widgets.fixed(df))
i_plot


Using matplotlib backend: QtAgg


interactive(children=(SelectMultiple(description='Msg hash', layout=Layout(height='100px', width='100%'), opti…